In [3]:
#install xml library
pip install lxml

SyntaxError: invalid syntax (<ipython-input-3-b3fa6f8df900>, line 2)

In [4]:
 #install request library
    pip install requests

IndentationError: unexpected indent (<ipython-input-4-d59b321760be>, line 2)

In [5]:
#import library
import lxml.html as lh
import pandas as pd
import requests
 import numpy as np 

In [6]:
#get url page content
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
tree =lh.fromstring(page.content)
tr_rows = tree.xpath('//tr')

In [7]:
#get table columns headings
col=[]
i=0

for t in tr_rows[0]:
    i+=1
    name=t.text_content()
    print  (i,name)
    col.append((name.rstrip('\n'),[]))

1 Postcode
2 Borough
3 Neighbourhood



In [8]:
#get table rows and create dataframe

for j in range(1,len(tr_rows)):
    T=tr_rows[j]
    if len(T)!=3:
         break
        #i is the index of our column
    i=0
     #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Append the data to the empty list of the i'th column
        col[i][1].append(data.rstrip('\n'))
        #Increment i for the next column
        i+=1
            
[len(C) for (title,C) in col]   
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [9]:
# if neighbourhood is not assigned set to borough
for n in df.index: 
  if df['Neighbourhood'][n] == 'Not assigned':
     df['Neighbourhood'][n] = df['Borough'][n]
       
#keep row not equal to Not assigned
df2 = df[df.Neighbourhood != 'Not assigned']

#reset df index
df2.reset_index(inplace = True)
df2.drop(columns='index', inplace = True)


/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [10]:
for r in df2.index:
   df2['Neighbourhood'][r] = df2['Neighbourhood'][r] + ", "

df2_grp = df2.groupby(['Postcode','Borough'], as_index = False)['Neighbourhood'].agg(sum) 

#remove trailing ", "
for x in df2_grp.index:
     df2_grp['Neighbourhood'][x] = df2_grp['Neighbourhood'][x][0:len(df2_grp['Neighbourhood'][x])-2]

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
df2_grp.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
df2_grp.shape

(103, 3)

In [15]:
#load geo codes, sort df and combine

Bour_lat = pd.read_csv("http://cocl.us/Geospatial_data")
df2_grp.sort_values(by='Postcode' , inplace=True)
Bour_lat.sort_values(by='Postal Code' , inplace=True)
Neigh = pd.merge(left=df2_grp,right=Bour_lat, left_on='Postcode', right_on='Postal Code')
Neigh.drop(columns= 'Postcode', inplace = True)
Neigh.head()


,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [1]:
#load needed libraries
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    altair-4.0.0               |             py_0         606 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be 

In [16]:
 #get neighborhoods with Toronto in thier name

Toronto_data = Neigh[Neigh['Borough'].str.contains('Toronto', na=False)]


In [17]:
#reset index and drop extra column
Toronto_data.reset_index(inplace = True)
Toronto_data.drop(columns= 'index', inplace = True)

In [18]:
#get toronto map.
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [19]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)


# add markers to map
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [20]:
#set client an secret for foursquare

CLIENT_ID = 'HLETKY3YLAR5D50MW4UALKXTCAT05PICK1KNGFTQAMQEJBQI' # your Foursquare ID
CLIENT_SECRET = 'R41ZCU3GRIU1Q2TSF5IKU3C5F2U51KEHM1H3ACHJEXD3GQD0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version



In [21]:
#create function to pull multiple venues

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
            
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                          'Neighborhood Latitude', 
                          'Neighborhood Longitude', 
	                      'Venue', 
	                      'Venue Latitude', 
	                      'Venue Longitude', 
	                      'Venue Category']
	        
    return(nearby_venues)

In [22]:
#get Toronto Venues
Toronto_venues = getNearbyVenues(names=Toronto_data['Postal Code'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

M4E
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6G
M6H
M6J
M6K
M6P
M6R
M6S
M7Y
M9A


In [23]:
#print shape and show head of venue

print(Toronto_venues.shape)
Toronto_venues.head()

print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

(1666, 7)
There are 233 uniques categories.


In [24]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
#group venues by neighborhood and re-index
Toronto_grped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grped.shape

(38, 233)

In [27]:
#create and display top 10 venues for each Neighborhood Postal code
num_top_venues = 10

for hood in Toronto_grped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grped[Toronto_grped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
                      venue  freq
0         Health Food Store  0.25
1                     Trail  0.25
2                       Pub  0.25
3               Yoga Studio  0.00
4                    Museum  0.00
5         Martial Arts Dojo  0.00
6            Massage Studio  0.00
7  Mediterranean Restaurant  0.00
8               Men's Store  0.00
9        Mexican Restaurant  0.00


----M4K----
                    venue  freq
0        Greek Restaurant  0.22
1             Coffee Shop  0.10
2          Ice Cream Shop  0.07
3      Italian Restaurant  0.05
4               Bookstore  0.05
5  Furniture / Home Store  0.05
6              Restaurant  0.05
7             Yoga Studio  0.02
8         Bubble Tea Shop  0.02
9                 Brewery  0.02


----M4L----
                venue  freq
0                Park  0.11
1                 Gym  0.05
2  Italian Restaurant  0.05
3         Pizza Place  0.05
4                 Pub  0.05
5       Movie Theater  0.05
6      Sandwich Place  0.05
7       Bu

In [29]:
#function that sorts venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
#create dataframe to display top ten venues

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grped['Neighborhood']

for ind in np.arange(Toronto_grped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Trail,Health Food Store,Pub,Wings Joint,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
1,M4K,Greek Restaurant,Coffee Shop,Ice Cream Shop,Restaurant,Bookstore,Italian Restaurant,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place
2,M4L,Park,Ice Cream Shop,Pizza Place,Pub,Fast Food Restaurant,Burrito Place,Italian Restaurant,Burger Joint,Fish & Chips Shop,Steakhouse
3,M4M,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Convenience Store,Sandwich Place,Brewery,Cheese Shop
4,M4N,Bus Line,Park,Swim School,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [34]:
#k-means to create 5 clusters

from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

array([0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [36]:


# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Toronto_merged = Toronto_data
    

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), how='inner',  on='Postal Code')


In [37]:
#create map with clusters
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], 
                                  Toronto_merged['Postal Code'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters



In [ ]:
#doesn't appear to be the best k vallue

In [41]:
#cluster groups

Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,0,Trail,Health Food Store,Pub,Wings Joint,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
1,"The Danforth West, Riverdale",0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Restaurant,Bookstore,Italian Restaurant,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place
2,"The Beaches West, India Bazaar",0,Park,Ice Cream Shop,Pizza Place,Pub,Fast Food Restaurant,Burrito Place,Italian Restaurant,Burger Joint,Fish & Chips Shop,Steakhouse
3,Studio District,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Convenience Store,Sandwich Place,Brewery,Cheese Shop
5,Davisville North,0,Park,Gym,Hotel,Clothing Store,Food & Drink Shop,Breakfast Spot,Sandwich Place,Discount Store,Dim Sum Restaurant,Diner
6,North Toronto West,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Grocery Store,Mexican Restaurant,Salon / Barbershop,Restaurant,Rental Car Location,Park,Miscellaneous Shop
7,Davisville,0,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Gym,Italian Restaurant,Café,Sushi Restaurant,Park,Brewery
9,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0,Coffee Shop,Pub,Vietnamese Restaurant,Fried Chicken Joint,Light Rail Station,Liquor Store,Sports Bar,Restaurant,Bagel Shop,Supermarket
11,"Cabbagetown, St. James Town",0,Coffee Shop,Restaurant,Bakery,Pub,Italian Restaurant,Pizza Place,Café,Park,Liquor Store,Japanese Restaurant
12,Church and Wellesley,0,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Sushi Restaurant,Yoga Studio,Café,Gym,Hotel,Mediterranean Restaurant


In [42]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Moore Park, Summerhill East",1,Playground,Tennis Court,Wings Joint,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [43]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Lawrence Park,2,Bus Line,Park,Swim School,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [44]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Roselawn,3,Ice Cream Shop,Garden,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [45]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Rosedale,4,Park,Playground,Trail,Wings Joint,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
23,"Forest Hill North, Forest Hill West",4,Park,Jewelry Store,Trail,Sushi Restaurant,Wings Joint,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
